In [ ]:
import datetime
import calendar
import pandas as pd
import os.path
import numpy as np

import time
import requests
currencies = ['BTC', 'ETH', 'LSK', 'GAME', 'BCC', 'LTC', 'DASH']
relays = ['PLN', 'EUR', 'USD', 'BTC']
period = 1.2

def check_bid_ask():
    diff_f = 1e-2
    diff_c = 1e-8

    pairs = {}

    for relay in relays:
        for c in currencies:
            if c != relay:
                url = 'https://bitbay.net/API/Public/' + c + relay + '/all.json'
                time.sleep(period)
                r = requests.get(url)
                r = r.json()
#                 print(r)
                if relay == 'BTC':
                    my_bid = round(r['bid']+diff_c,8)
                    my_ask = round(r['ask']-diff_c,8)
                else:
                    my_bid = round(r['bid']+diff_f,2)
                    my_ask = round(r['ask']-diff_f,2)
                pairs[c+relay] = {'bid': r['bid'], 'ask': r['ask'], 'my_bid': my_bid, 'my_ask': my_ask}
#                 print(pairs[c1+c2])
    return pairs

# pairs = check_bid_ask()


In [ ]:
def transform_to_BTC_value(pairs):
    offerts_BTC = {}

    for c in currencies:
        offerts_bid = {}
        offerts_ask = {}
        for relay in ['PLN', 'EUR', 'USD']:
            offerts_bid['bid_'+relay] = pairs[c+relay]['my_bid']/pairs['BTC'+relay]['my_ask']
            offerts_ask['ask_'+relay] = pairs[c+relay]['my_ask']/pairs['BTC'+relay]['my_bid']
        if c != 'BTC':
            offerts_bid['bid_BTC'] = pairs[c+'BTC']['my_bid']
            offerts_ask['ask_BTC'] = pairs[c+'BTC']['my_ask']
        offerts_BTC[c] = {'bid': offerts_bid, 'ask': offerts_ask}
    
    return offerts_BTC 

# offerts_BTC = transform_to_BTC_value(pairs)


def plot_currency_BTC_value(offerts_BTC):
    matplotlib.rcParams['figure.figsize'] = (18,4)
    for key in offerts_BTC.keys():
        a = {"x":[], "y":[], "label":[]}
        for label, coord in offerts_BTC[key]['ask'].items():
            a["x"].append(coord)
            a["y"].append(0)
            a["label"].append(label)

        b = {"x":[], "y":[], "label":[]}
        for label, coord in offerts_BTC[key]['bid'].items():
            b["x"].append(coord)
            b["y"].append(0)
            b["label"].append(label)

        # display scatter plot data
        plt.title(key, fontsize=20)
        plt.xlim([min(b["x"]),max(a["x"])])    
        plt.scatter(a["x"], a["y"], marker = 'o', color = 'b', s = 100)
        plt.scatter(b["x"], b["y"], marker = 'o', color = 'r', s = 100)

        for label, x in zip(a["label"], a["x"]):
            plt.annotate(label, xy = (x, 0))

        for label, x in zip(b["label"], b["x"]):
            plt.annotate(label, xy = (x, 0))

        plt.show()
        
# plot_currency_BTC_value(offerts_BTC)

In [ ]:
def find_profitable_links(offerts_BTC):
    fee = 0.0037
    threshold = 0.05
    gap = []
    for c in currencies:
        for relay in relays:
            if(c != relay):
                g = offerts_BTC[c]['ask']['ask_'+relay]/offerts_BTC[c]['bid']['bid_'+relay]*(1-2*fee)-1
                gap.append([c+relay,c,relay,g])

    gap_sorted = sorted(gap, key=lambda x : -x[3])
    gap = [x for x in gap_sorted if x[3] > threshold]
    return gap
   
# gap = find_profitable_links(offerts_BTC)
# # print(len(gap_sorted))
# print(gap)
# print(len(gap))

# for c in currencies:
#     print(c, len([x for x in gap if x[1] == c]))
    
# for relay in relays:
#     print(relay, len([x for x in gap if x[2] == relay]))
 

In [ ]:
import requests
import urllib
import hmac
import hashlib
import json

# import ntplib
# from time import ctime
# c = ntplib.NTPClient()
# t = c.request('pool.ntp.org').tx_time
# print(t)
# print(int(time.time()))

def info():
    time.sleep(period)
    q = {'method': 'info'}
    return request(q)

def info():
    time.sleep(period)
    q = {'method': 'info'}
    return request(q)

def orderbook(order_currency, payment_currency):
    time.sleep(period)
    q = {'method': 'orderbook', 'order_currency': order_currency, 'payment_currency': payment_currency}
    return request(q)

def bid(currency, amount, payment_currency, rate):
    time.sleep(period)
    q = {'method': 'trade', 'type': 'bid', 'currency': currency , 'amount': amount, 'payment_currency': payment_currency, 'rate': rate}
    """return: order_id, error"""
    return request(q)

def ask(currency, amount, payment_currency, rate):
    time.sleep(period)
    print('ask', currency, amount, payment_currency, rate)
    q = {'method': 'trade', 'type': 'ask', 'currency': currency , 'amount': amount, 'payment_currency': payment_currency, 'rate': rate}
    """return: order_id, error"""
    return request(q)

def cancel(id):
    time.sleep(period)
    q = {'method': 'cancel', 'id': id}
    """return: success, error"""
    return request(q)

def cancel_all():
    o = current_orders()
    for order in o:
        cancel(order["order_id"])

def current_orders(limit = 100):
    time.sleep(period)
    q = {'method': 'orders', 'limit': limit}
    """return: order_id, order_currency, order_date, payment_currency, type, status, units, start_units, current_price, start_price"""
    print(request(q))
    return request(q)

def transactions(currency1, currency2):
    time.sleep(period)
    q = {'method': 'transactions', 'market': currency1+'-'+currency2}
    """return: date, type, market, amount, rate, price"""
    return request(q)

def print_json(js):
    print(json.dumps(js, indent=2))
    
def request(query):
    
    
    API_key = b'cef234b2-7a8b-4621-94b4-228c45bfa072'
    API_secret = b'f1699871-d42e-476d-a86e-ac65fdf0bf89'
    url = 'https://bitbay.net/API/Trading/tradingApi.php'
    
    for attempt in range(5):
        try:
            query['moment'] = int(time.time())
            post = urllib.parse.urlencode(query)
            API_Hash = hmac.new(API_secret, post.encode('utf-8'), hashlib.sha512)
            headers = {'API-Key': API_key, 'API-Hash': API_Hash.hexdigest()}
    
            answer = requests.post(url, data=post, headers=headers)
#             print(answer)
#             print(answer.json())
            if 'code' in answer.json():
                print('code in answer')
                raise
        except:
            time.sleep(period)
        else:
            break
    else:
        print('Proba polaczenia nieudana')
        
    return answer.json()

In [ ]:
def print_wallet(wallet):
    sum = 0
    for c in wallet['balances'].keys():
        a = float(wallet['balances'][c]['available'])
        l = float(wallet['balances'][c]['locked'])
        s = round(a + l, 8)

        try:
            print(c, a, l, s, round(s*offerts_BTC[c]['ask']['ask_BTC']*1000, 1))
            sum += round(s*offerts_BTC[c]['ask']['ask_BTC'], 8)
        except:
            print(c, a, l, s, s)
            
def calculate_wallet(wallet):
    minimum = {}
    total_BTC = 0
    extended_wallet = {}
    for c in currencies:
        a = float(wallet['balances'][c]['available'])
        l = float(wallet['balances'][c]['locked'])
        s = round(a + l, 8)

        extended_wallet[c] = {'available': a, 'locked': l, 'sum': s}
            
    for r in relays:
        if r != 'BTC':
            a = float(wallet['balances'][r]['available'])
            l = float(wallet['balances'][r]['locked'])
            s = round(a + l, 2)
            extended_wallet[r] = {'available': a, 'locked': l, 'sum': s}
    
    return extended_wallet

In [ ]:
def sell_currency(currency, f='ask'):
    print('-------------')
    print(currency)
    print()
    links = len([x for x in gap if x[1] == currency])
    print('links', links)
    
    if links > 0:
        for relay in relays:
            if len([x for x in gap if x[0] == currency+relay]) == 1:
                deposit = float(wallet['balances'][currency]['available'])+float(wallet['balances'][currency]['locked'])
                amount = round(deposit/links/4,8)

                if currency+relay+f in orders:
                    print(orders[currency+relay+f]['order_id'])
                    if len([item for item in o if item["order_id"] == orders[currency+relay+f]['order_id']]) == 1:                   
                        if pairs[currency+relay][f] != orders[currency+relay+f]['rate']:
                            cancel(orders[currency+relay+f]['order_id'])
                            del orders[currency+relay+f]
                            print('Ktoś dał lepszą ofertę')
                        else:
                            print("Oferta istnieje")
                    else:
                        del orders[currency+relay+'ask']
                        print('Prawdopodobnie udana transakcja')
                else:
                    if amount > 0.0001:
                        t = ask(currency,amount,relay,pairs[currency+relay]['my_'+f])
                        orders[currency+relay+f] = t
                        print(t)
                        print(time.time(), "Oferta wystawiona ", orders[currency+relay+f]['order_id'])
                    else:
                        print(time.time(), "Brak wolnych środków")
            else:
                if currency+relay+f in orders:
                    print(cancel(orders[currency+relay+f]['order_id']))
                    del orders[currency+relay+f]
                print("Niezyskowne")
                
def buy_currency(relay, f='bid'):
    print('-------------')
    print(relay)
    print()
    links = len([x for x in gap if x[2] == relay])
    
    if links > 0:
        for currency in currencies:
            if len([x for x in gap if x[0] == currency+relay]) == 1:  
                deposit = float(wallet['balances'][currency]['available'])+float(wallet['balances'][currency]['locked'])
                amount = deposit/links             

                if currency+relay+f in orders:
                    if len([item for item in o if item["order_id"] == orders[currency+relay+f]['order_id']]) == 1:                        
                        if pairs[currency+relay][f] != orders[currency+relay+f]['rate']:
                            cancel(orders[currency+relay+f]['order_id'])
                            del orders[currency+relay+f]
                            print('Ktoś dał lepszą ofertę')
                        else:
                            print("Oferta istnieje")
                    else:
                        del orders[currency+relay+f]
                        print('Prawdopodobnie udana transakcja')
                else:
                    if relay == 'BTC':
                        thres = 0.0001
                        amount = amount/8
                    else:
                        thres = 1.0
                    if amount > thres:
                        amount = round(amount/pairs[currency+relay]['my_'+f],8) 
                        t = bid(currency,amount,relay,pairs[currency+relay]['my_'+f])                            
                        orders[currency+relay+f] = t
                        print(t)
                        print(time.time(), "Oferta wystawiona ", orders[currency+relay+f]['order_id'])
                    else:
                        print(time.time(), "Brak wolnych środków")
            else:
                if currency+relay+f in orders:
                    print(cancel(orders[currency+relay+f]['order_id']))
                    del orders[currency+relay+f]
                print("Niezyskowne")

# print_json(orderbook('BTC', 'PLN'))

# print(cancel_all())
# pairs = check_bid_ask()
# print(calculate_wallet(info(), pairs))

# orders = {}
# while True:
#     pairs = check_bid_ask()
#     offerts_BTC = transform_to_BTC_value(pairs)
#     plot_currency_BTC_value(offerts_BTC)
#     gap = find_profitable_links(offerts_BTC)
#     print(len(gap))
#     wallet = info()
#     print_wallet(wallet)
#     o = current_orders()
    
#     for currency in currencies:
#         sell_currency(currency) 
#     for relay in relays:
#         buy_currency(relay)



In [ ]:
import pandas as pd
import time
import os.path
import requests
from shutil import copyfile
import cProfile
import datetime
import talib

In [ ]:
def resample_df(df, frequency = '1H', window1 = 6, window2 = 40):
    one_hot = pd.get_dummies(df['type'])
    df = df.join(one_hot)

    df_pv = df.resample(frequency).agg({'price': np.mean, 'amount': np.sum, 'buy': np.sum, 'sell': np.sum})
    df_ohlc = df['price'].resample(frequency).ohlc()

    df = pd.concat([df_pv, df_ohlc], axis=1)
    df = df.fillna(method='pad')
    df = df.dropna()
    
    df['SMA(24)'] = df['price'].rolling(24).mean()
    df['SMA(12)'] = df['price'].rolling(12).mean()
    
    return df


def download_trades(pair='BTCPLN', since=0):
    url = 'https://bitbay.net/API/Public/' + pair + '/trades.json?since=' + str(since)
    time.sleep(min_period)
    r = requests.get(url)
#     print(r.json())
    return r.json()

def update_trades(pair, since):
    trades_file = 'Data/trades_BitBay_' + pair + '.csv'
    
    try:
        trades = download_trades(pair=pair, since=since)
    except (KeyboardInterrupt, SystemExit):
        raise
    except:
        print(pair, "except")
        pass
    else:
        if len(trades) > 0:
            df_temp = pd.DataFrame(data=trades, columns=['date','price','type','amount','tid'])
            df_temp[['amount','price']] = df_temp[['amount','price']].astype('float64')
            df_temp.index = df_temp['tid'].values

            try:
                copyfile(trades_file, trades_file+'temp')
                df_temp.to_csv(trades_file+'temp', mode='a', header=0)
                os.replace(trades_file+'temp', trades_file)
                copyfile(trades_file, trades_file+'bac')
                since = df_temp['tid'].iloc[-1]
            except:
                print(pair, "Problem zapisu")
                copyfile(trades_file+'bac', trades_file)          

            print(pair, pd.to_datetime(df_temp['date'], unit='s').tail(1), df_temp.tail(1))
            
    return since, df_temp

In [ ]:
period = 10
min_period = 1.05
sinces = {}

cooldown = datetime.timedelta(seconds=5)
min_PLN = 1e-1
min_BTC = 1e-6

buysell = 0.000
n = 4
limit_sell = buysell
limit_buy = buysell

frequency = '5min'
currency = 'BTC'
relay = 'PLN'
pair = currency + relay
trades_file = 'Data/trades_BitBay_' + pair + '.csv'

df_trades = pd.read_csv(trades_file, index_col=0, usecols=['date','price', 'type', 'amount'], header=0, engine='python')
sinces[pair] = df_trades['tid'].iloc[-1]

while True:  
    sinces[pair], df_temp = update_trades(pair=pair, since=sinces[pair])
    df_trades = df_trades.append(df_temp, ignore_index=True)
    dataframe = df_trades
    dataframe.index = pd.to_datetime(dataframe.index, unit='s')
    trades_ohlc = resample_df(dataframe, frequency)

    wallet = calculate_wallet(info())
    PLN = wallet[relay]['sum']
    BTC = wallet[currency]['sum']

    window = trades_ohlc.iloc[-(n+1):]
    average = trades_ohlc['SMA(12)'][-1]
    print(trades_ohlc.index[-1])

    mfi = talib.MFI(window['high'].values, window['low'].values, window['close'].values, window['amount'].values, n)[-1]
    cci = talib.CCI(window['high'].values, window['low'].values, window['close'].values, n)[-1]
    cmo = talib.CMO(window['close'].values, n)[-1]

    if cci > 100 and mfi > 80 and cmo > 50:
        sell = True
    else:
        sell = False

    if cci < -100 and mfi < 20 and cmo < -50:
        buy = True
    else:
        buy = False

    buy_price = average*(1+limit_buy)        
    if(buy and BTC > min_BTC):
        print('ask', buy_price)
        cancel_all()
        wallet = calculate_wallet(info())
    #     t = ask(currency, wallet[currency]['available'], relay, round(buy_price, 2))
    #     print(time.time(), "Oferta wystawiona ASK ", t)

    sell_price = average*(1-limit_sell)
    if(sell and PLN > min_PLN):
        print('bid', sell_price)
        cancel_all()
        wallet = calculate_wallet(info())
    #     t = bid(currency, round(wallet[relay]['available']/sell_price, 8), relay, round(sell_price, 2))                            
    #     print(time.time(), "Oferta wystawiona BID ", t)
    
        

In [ ]:
import gdax
import pandas as pd
import time
import os.path
import requests
from shutil import copyfile
import cProfile
import datetime
import talib

n = 4
public_client = gdax.PublicClient()

buy = False
sell = False
bidding = False
asking = False

PLN = 0 
BTC = 1
PLN_min = 1e-1
BTC_min = 1e-6

cci_thr = 95
mfi_thr = 0
cmo_thr = 10

API_p = '557c4731915037e278adcb3aa96c029a'
API_s = 'nt/cMZQ5rvxlDrK8qhw2wtqqZD16q3VyINlTpG6+Ba8pB5SsMWpS7VcyROPqZrFjAbD/dwEi0ucNsSCWGBzOsg=='

In [ ]:
while True:    
    for attempt in range(5):
        try:
            historic_ohlc = public_client.get_product_historic_rates('BTC-EUR', granularity=300)
        except:
            time.sleep(0.25)
        else:
            break
    else:
        print('Proba polaczenia nieudana')
        
    trades_ohlc = pd.DataFrame(data=historic_ohlc, columns=['date','low','high','open','close','size'])
    trades_ohlc.index = trades_ohlc['date'].values
    trades_ohlc = trades_ohlc.sort_index()
    
    window = trades_ohlc.iloc[-(n+1):]
    average = trades_ohlc['close'].iloc[-1]
    
    mfi = talib.MFI(window['high'].values, window['low'].values, window['close'].values, window['size'].values, n)[-1]
    cci = talib.CCI(window['high'].values, window['low'].values, window['close'].values, n)[-1]
    cmo = talib.CMO(window['close'].values, n)[-1]

    
    equilibrium = PLN/(PLN+BTC*average)
                            
    if cci > cci_thr and mfi > mfi_thr and cmo > cmo_thr:
        sell_new = True
    else:
        sell_new = False

    if cci < -cci_thr and mfi < 100 - mfi_thr and cmo < -cmo_thr:
        buy_new = True
    else:
        buy_new = False

#     if(equilibrium > 0.01 and equilibrium < 0.99):
#         if(buy and not buy_new and not sell_new):
#             buy_new = True
#         elif(sell and not sell_new and not buy_new):
#             sell_new = True

    buy = buy_new
    sell = sell_new
    
    for attempt in range(5):
        try:
            orderbook_l1 = public_client.get_product_order_book('BTC-EUR', level=1)
        except:
            time.sleep(0.25)
        else:
            break
    else:
        print('Proba polaczenia nieudana')
    

    bids = orderbook_l1['bids'][0]
    asks = orderbook_l1['asks'][0]
    best_bid = float(bids[0])
    best_ask = float(asks[0])
    difference = round(best_ask - best_bid, 2)

    if bidding and bid != best_bid:
        if bid > best_bid:
            BTC = PLN/bid
            PLN = 0
        print('remove bid')
        bidding = False
    elif bidding:
        if queue - float(bids[1]) > 0:
            queue -= float(bids[1])
        if queue < 0:
            BTC = PLN/bid
            PLN = 0
            print('remove bid')
            bidding = False
        
    if(asking and ask != best_ask):
        if(ask < best_ask):
            PLN = BTC*ask
            BTC = 0
        print('remove ask')
        asking = False
    elif asking:
        if queue - float(asks[1]) > 0:
            queue -= float(asks[1])
        if queue < 0:
            PLN = BTC*ask
            BTC = 0
            print('remove ask')
            asking = False

    if(buy and not bidding and PLN > PLN_min):
        if(difference != 0.01):
            print('buy', best_bid + 0.01)    
            bid = best_bid + 0.01
            queue = PLN/average
        else:
            print('buy', best_bid)
            bid = best_bid
            queue = float(bids[1])
        bidding = True
        

    if(sell and not asking and BTC > BTC_min):
        if(difference != 0.01):
            print('sell', best_ask - 0.01)
            ask = best_ask - 0.01
            queue = BTC
            queue_my = queue
        else:
            print('sell', best_ask)       
            ask = best_ask
            queue = float(asks[1])
            queue_my = queue + BTC
        asking = True
        

    print(orderbook_l1)
    print(PLN, BTC)
    print()

    time.sleep(5)

{'asks': [['8633.39', '1.21725859', 4]], 'sequence': 3419037571, 'bids': [['8633.38', '2.77996687', 21]]}
0 1

{'asks': [['8634', '0.33844272', 1]], 'sequence': 3419037855, 'bids': [['8633.99', '2.52400531', 9]]}
0 1

{'asks': [['8634', '0.37214272', 2]], 'sequence': 3419038154, 'bids': [['8633.99', '2.69525168', 13]]}
0 1

{'asks': [['8640', '1.12829416', 1]], 'sequence': 3419038515, 'bids': [['8639.99', '2.8885182', 11]]}
0 1

{'asks': [['8640', '3.46700911', 5]], 'sequence': 3419038752, 'bids': [['8639.99', '3.79451404', 13]]}
0 1

{'asks': [['8640', '3.30993367', 6]], 'sequence': 3419039166, 'bids': [['8639.99', '4.0333704', 17]]}
0 1

{'asks': [['8640', '3.30993367', 6]], 'sequence': 3419039432, 'bids': [['8639.99', '4.54454314', 18]]}
0 1

{'asks': [['8640', '3.30993367', 6]], 'sequence': 3419039532, 'bids': [['8639.99', '2.43507718', 13]]}
0 1

{'asks': [['8640', '3.30993367', 6]], 'sequence': 3419039668, 'bids': [['8639.99', '2.43507718', 13]]}
0 1

{'asks': [['8640', '3.239933

{'asks': [['8632.04', '4.64617034', 7]], 'sequence': 3419063795, 'bids': [['8632.03', '4.90337946', 22]]}
0 1

{'asks': [['8632.04', '4.62617034', 7]], 'sequence': 3419064075, 'bids': [['8632.03', '5.07127411', 22]]}
0 1

{'asks': [['8632.04', '4.54617034', 7]], 'sequence': 3419064244, 'bids': [['8632.03', '5.07127411', 22]]}
0 1

{'asks': [['8632.04', '4.54617034', 7]], 'sequence': 3419064558, 'bids': [['8632.03', '4.09725887', 19]]}
0 1

{'asks': [['8632.04', '4.54617034', 7]], 'sequence': 3419064736, 'bids': [['8632.03', '3.92386367', 18]]}
0 1

{'asks': [['8632.04', '4.54617034', 7]], 'sequence': 3419065069, 'bids': [['8632.03', '3.04357718', 15]]}
0 1

{'asks': [['8632.04', '5.74766713', 8]], 'sequence': 3419065364, 'bids': [['8632.03', '4.37617745', 16]]}
0 1

{'asks': [['8632.04', '5.70366713', 9]], 'sequence': 3419065763, 'bids': [['8632.03', '3.4697565', 12]]}
0 1

{'asks': [['8632.04', '5.51766713', 8]], 'sequence': 3419066021, 'bids': [['8632.03', '3.53495595', 13]]}
0 1

{'

{'asks': [['8710', '4.24634581', 8]], 'sequence': 3419097568, 'bids': [['8709.99', '0.24964999', 5]]}
8684.0 0

{'asks': [['8710', '2.15934824', 5]], 'sequence': 3419097767, 'bids': [['8709.99', '0.46364999', 6]]}
8684.0 0

{'asks': [['8710', '4.23634581', 8]], 'sequence': 3419098032, 'bids': [['8709.99', '0.24964999', 5]]}
8684.0 0

{'asks': [['8710', '4.21197248', 9]], 'sequence': 3419098205, 'bids': [['8709.99', '0.24964999', 5]]}
8684.0 0

{'asks': [['8710', '5.20233722', 10]], 'sequence': 3419098386, 'bids': [['8709.99', '0.92186603', 7]]}
8684.0 0

{'asks': [['8710', '1.24419772', 3]], 'sequence': 3419098637, 'bids': [['8709.99', '2.29281114', 11]]}
8684.0 0

{'asks': [['8710', '0.14083298', 1]], 'sequence': 3419098793, 'bids': [['8709.99', '2.29281114', 11]]}
8684.0 0

{'asks': [['8710', '0.45562298', 2]], 'sequence': 3419098931, 'bids': [['8709.99', '2.29281114', 11]]}
8684.0 0

{'asks': [['8710', '0.08983298', 1]], 'sequence': 3419099160, 'bids': [['8709.99', '2.29281114', 11]

{'asks': [['8701.06', '0.31186472', 1]], 'sequence': 3419120609, 'bids': [['8701.05', '0.31139567', 2]]}
8684.0 0

{'asks': [['8701.06', '4.1257397', 3]], 'sequence': 3419120996, 'bids': [['8701.05', '1.94777327', 6]]}
8684.0 0

{'asks': [['8701.06', '6.24757509', 5]], 'sequence': 3419121285, 'bids': [['8701.05', '1.94364532', 6]]}
8684.0 0

{'asks': [['8701.06', '6.16757509', 5]], 'sequence': 3419121822, 'bids': [['8701.05', '0.88811143', 5]]}
8684.0 0

{'asks': [['8701.06', '7.61276359', 6]], 'sequence': 3419122141, 'bids': [['8701.05', '0.51180022', 4]]}
8684.0 0

{'asks': [['8701.06', '8.12999358', 8]], 'sequence': 3419122895, 'bids': [['8701.05', '0.18586971', 2]]}
8684.0 0

{'asks': [['8700.01', '2.06389232', 2]], 'sequence': 3419123435, 'bids': [['8700', '1.40931036', 2]]}
8684.0 0

{'asks': [['8700.01', '3.56078989', 5]], 'sequence': 3419124288, 'bids': [['8700', '0.28009344', 1]]}
8684.0 0

{'asks': [['8700.01', '7.52808647', 11]], 'sequence': 3419124757, 'bids': [['8700', '1.

{'asks': [['8684.5', '5.86372101', 6]], 'sequence': 3419147647, 'bids': [['8684.49', '2.15327927', 7]]}
8684.0 0

{'asks': [['8684.5', '3.60174581', 4]], 'sequence': 3419147981, 'bids': [['8684.49', '2.09427927', 7]]}
8684.0 0

{'asks': [['8684.5', '2.43974581', 3]], 'sequence': 3419148309, 'bids': [['8684.49', '2.47302327', 10]]}
8684.0 0

{'asks': [['8684.5', '2.40974581', 3]], 'sequence': 3419148575, 'bids': [['8684.49', '2.46302327', 9]]}
8684.0 0

{'asks': [['8684.5', '1.80301947', 2]], 'sequence': 3419148922, 'bids': [['8684.49', '3.41357603', 13]]}
8684.0 0

{'asks': [['8684.5', '1.80301947', 2]], 'sequence': 3419149215, 'bids': [['8684.49', '3.11357603', 12]]}
8684.0 0

{'asks': [['8684.5', '1.80301947', 2]], 'sequence': 3419149575, 'bids': [['8684.49', '3.41357603', 13]]}
8684.0 0

{'asks': [['8684.5', '1.80301947', 2]], 'sequence': 3419150026, 'bids': [['8684.49', '2.84484801', 12]]}
8684.0 0

{'asks': [['8684.5', '1.78301947', 2]], 'sequence': 3419150344, 'bids': [['8684.49'

...

{'asks': [['8483.87', '1.1646234', 5]], 'sequence': 3423102140, 'bids': [['8483.86', '0.55041', 5]]}
0 0.9987205997993094

sell 8483.87
{'asks': [['8483.87', '1.1646234', 5]], 'sequence': 3423102290, 'bids': [['8483.86', '0.55041', 5]]}
0 0.9987205997993094

{'asks': [['8483.87', '1.1646234', 5]], 'sequence': 3423102483, 'bids': [['8483.86', '0.55041', 5]]}
0 0.9987205997993094

{'asks': [['8483.87', '1.1646234', 5]], 'sequence': 3423102763, 'bids': [['8483.86', '0.48004', 4]]}
0 0.9987205997993094

{'asks': [['8483.87', '1.1646234', 5]], 'sequence': 3423102919, 'bids': [['8483.86', '0.48004', 4]]}
0 0.9987205997993094

{'asks': [['8483.87', '1.1646234', 5]], 'sequence': 3423103030, 'bids': [['8483.86', '0.55041', 5]]}
0 0.9987205997993094

{'asks': [['8483.87', '1.1646234', 5]], 'sequence': 3423103341, 'bids': [['8483.86', '0.55041', 5]]}
0 0.9987205997993094

{'asks': [['8483.87', '0.91578732', 4]], 'sequence': 3423103717, 'bids': [['8483.86', '0.55041', 5]]}
0 0.9987205997993094

{'

{'asks': [['8490.27', '1.59719273', 2]], 'sequence': 3423114757, 'bids': [['8490.26', '3.21544395', 14]]}
8473.015735019368 0

{'asks': [['8490.27', '1.59919273', 3]], 'sequence': 3423114984, 'bids': [['8490.26', '4.37877852', 16]]}
8473.015735019368 0

{'asks': [['8490.27', '0.51183272', 3]], 'sequence': 3423115122, 'bids': [['8490.26', '3.21544395', 14]]}
8473.015735019368 0

{'asks': [['8490.87', '0.10463', 1]], 'sequence': 3423115520, 'bids': [['8490.86', '2.29615863', 11]]}
8473.015735019368 0

{'asks': [['8490.87', '0.10463', 1]], 'sequence': 3423115844, 'bids': [['8490.86', '2.95924965', 13]]}
8473.015735019368 0

{'asks': [['8490.87', '0.10463', 1]], 'sequence': 3423116142, 'bids': [['8490.86', '2.97457152', 14]]}
8473.015735019368 0

{'asks': [['8490.87', '0.10463', 1]], 'sequence': 3423116366, 'bids': [['8490.86', '4.16295639', 16]]}
8473.015735019368 0

{'asks': [['8506.91', '1.23101441', 1]], 'sequence': 3423116746, 'bids': [['8506.9', '0.11', 1]]}
8473.015735019368 0

{'as

{'asks': [['8506.91', '6.82193297', 17]], 'sequence': 3423129453, 'bids': [['8506.9', '1.65204915', 11]]}
8473.015735019368 0

{'asks': [['8506.91', '2.65260489', 14]], 'sequence': 3423129762, 'bids': [['8506.9', '2.40204915', 13]]}
8473.015735019368 0

{'asks': [['8506.91', '2.52635939', 13]], 'sequence': 3423130190, 'bids': [['8506.9', '2.72078915', 15]]}
8473.015735019368 0

{'asks': [['8506.91', '1.88835939', 11]], 'sequence': 3423130368, 'bids': [['8506.9', '2.72078915', 15]]}
8473.015735019368 0

{'asks': [['8506.91', '1.21907941', 7]], 'sequence': 3423130800, 'bids': [['8506.9', '2.80564238', 17]]}
8473.015735019368 0

{'asks': [['8506.91', '0.85539877', 6]], 'sequence': 3423131083, 'bids': [['8506.9', '2.80564238', 17]]}
8473.015735019368 0

{'asks': [['8506.91', '0.86990155', 7]], 'sequence': 3423131494, 'bids': [['8506.9', '2.33543585', 16]]}
8473.015735019368 0

{'asks': [['8506.91', '0.85990155', 7]], 'sequence': 3423131757, 'bids': [['8506.9', '3.99846833', 19]]}
8473.0157

{'asks': [['8542.4', '6.52183776', 10]], 'sequence': 3423148102, 'bids': [['8542.39', '0.66881258', 9]]}
8473.015735019368 0

{'asks': [['8542.4', '6.15815712', 10]], 'sequence': 3423148363, 'bids': [['8542.39', '0.66881258', 9]]}
8473.015735019368 0

{'asks': [['8542.4', '6.33220077', 8]], 'sequence': 3423148568, 'bids': [['8542.39', '0.97856258', 11]]}
8473.015735019368 0

{'asks': [['8542.4', '6.33220077', 8]], 'sequence': 3423148758, 'bids': [['8542.39', '1.65828328', 13]]}
8473.015735019368 0

{'asks': [['8542.4', '6.33220077', 8]], 'sequence': 3423148918, 'bids': [['8542.39', '1.65828328', 13]]}
8473.015735019368 0

{'asks': [['8542.4', '6.33220077', 8]], 'sequence': 3423149075, 'bids': [['8542.39', '1.07322026', 11]]}
8473.015735019368 0

{'asks': [['8542.4', '6.48789712', 9]], 'sequence': 3423149261, 'bids': [['8542.39', '1.07322026', 11]]}
8473.015735019368 0

{'asks': [['8542.4', '3.96900857', 8]], 'sequence': 3423149596, 'bids': [['8542.39', '1.08614849', 13]]}
8473.01573501

{'asks': [['8542.4', '2.90463592', 5]], 'sequence': 3423158489, 'bids': [['8542.39', '1.45480509', 15]]}
8473.015735019368 0

{'asks': [['8542.4', '3.84321768', 6]], 'sequence': 3423158723, 'bids': [['8542.39', '1.45480509', 15]]}
8473.015735019368 0

{'asks': [['8542.4', '3.62343036', 5]], 'sequence': 3423158967, 'bids': [['8542.39', '1.53445123', 16]]}
8473.015735019368 0

{'asks': [['8542.4', '3.84321768', 6]], 'sequence': 3423159118, 'bids': [['8542.39', '1.45480509', 15]]}
8473.015735019368 0

{'asks': [['8542.4', '3.62343036', 5]], 'sequence': 3423159287, 'bids': [['8542.39', '1.53445123', 16]]}
8473.015735019368 0

{'asks': [['8542.4', '2.98543036', 3]], 'sequence': 3423159448, 'bids': [['8542.39', '1.90270425', 16]]}
8473.015735019368 0

{'asks': [['8542.4', '2.98543036', 3]], 'sequence': 3423159664, 'bids': [['8542.39', '2.13138034', 17]]}
8473.015735019368 0

{'asks': [['8542.4', '3.20521768', 4]], 'sequence': 3423159881, 'bids': [['8542.39', '2.0517342', 16]]}
8473.015735019